In [1]:
import pandas as pd
import numpy as np
from pythainlp.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import pickle
import csv

In [2]:
f = open("./train.txt", "r")
x_train = np.array([word_tokenize(line.strip(),'newmm') for line in f.readlines()])
f.close()
f = open("./train_label.txt", "r")
y_train = pd.Series([line.strip() for line in f.readlines()])
f.close()
x_train.shape,y_train.shape

((24345,), (24345,))

In [3]:
label=pd.factorize(y_train)[1]
print(label)

Index(['neu', 'neg', 'pos', 'q'], dtype='object')


In [4]:
parameters = [
    #{'C': [10, 100, 1000], 'kernel': ['linear']},
    #{'C': [100], 'gamma': [0.1,0.01], 'degree':[2,3],'kernel': ['poly'],'decision_function_shape':['ovr']},
    {'C': [100], 'gamma': [0.01], 'degree':[2], 'kernel': ['rbf'],'decision_function_shape':['ovr']}
             ]

In [5]:
f = open("./test.txt", "r")
x_test = np.array([word_tokenize(line.strip(),'newmm') for line in f.readlines()])
f.close()

In [6]:
dict_size = 7700
vectorizer = TfidfVectorizer(max_features=dict_size,analyzer='char_wb',ngram_range=(1, 3))
X = vectorizer.fit_transform([' '.join(i) for i in x_train])
label=list(pd.factorize(y_train)[1])
Y=[label.index(i) for i in y_train]

svc = svm.SVC(cache_size=900)
clf = GridSearchCV(svc, parameters, cv=10,n_jobs=-1)
clf.fit(X, Y)

y_pred = clf.predict(X)

acc = accuracy_score(y_pred,Y)
print(acc,dict_size)

test_data = vectorizer.transform([' '.join(i) for i in x_test])
output = clf.predict(test_data)
y_pred=[]
for i in output:
    y_pred.append(label[i])

with open('file_'+str(dict_size)+'.csv', mode='w') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['Id','Class'])
    [writer.writerow([i+1,j]) for i,j in enumerate(y_pred)]

filename = 'SVM_'+str(dict_size)+'.sav'
pickle.dump(clf, open(filename, 'wb'))
#del X,Y,y_pred,output,acc,clf,svc,label,vectorizer,filename

0.7710823577736702 7700
